In [58]:
text = "WHAT DID YOU SAY"
text = text.lower()
print(text)
import random
#from ipynb.fs.full.zoecode import *
import re
from math import log
from math import isclose
import numpy

what did you say


alpha_list is just a list of all the letters in the alphabet with a space included. This helps assign each letter an index and is used in many definitions.

In [59]:
alpha_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
             'p','q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

The function char_to_num, takes a piece of text (a string of lowercase letters) and returns a list of the numerical values for each number, based on the indices in alpha_list. 

The function after that, num_to_char, reverses the effect, going from a series of numbers (ranged 0 - 26) to a string of lowercase letters returned as a list.

In [60]:
def char_to_num(char_text):
    num_text = []
    for letter in char_text:
        num_text.append(alpha_list.index(letter))
    return num_text

In [61]:
def num_to_char(num_text):
    char_text = []
    for num in num_text:
        char_text.append(alpha_list[num])
    return char_text

A caesar cipher is just a shift cipher.
encode_caesar_cipher takes a piece of text and a key, and shifts each letter forwards by the key. For example, if the key was 3, an 'a' would become a 'd', a 'b' would become a 'e' , and so on. decode_caesar_cipher works much the same way, but instead of shifting the letters forwards, it shifts them backwards, effectively reversing the encoding of the cipher. This only works if you use the same key to encode and decode, however.

In [62]:
def encode_caesar_cipher(plaintext, key):
    ciphertext_num = []
    plaintext_num = char_to_num(plaintext)
    for num in plaintext_num:
        if num != 26: #num 26 is a space
            num += key
            num = num % 26
        ciphertext_num.append(num)
    ciphertext = num_to_char(ciphertext_num)
    return ciphertext

In [63]:
def decode_caesar_cipher(ciphertext, key):
    plaintext_num = []
    ciphertext_num = char_to_num(ciphertext)
    for num in ciphertext_num:
        if num != 26: #num 26 is a space
            num -= key
            num = num % 26
        plaintext_num.append(num)
    plaintext = num_to_char(plaintext_num)
    return plaintext

An aristocrat cipher is a random substitution cipher. Each letter in the alphabet encodes to another letter. In this code, I often abbreviate aristocrat to 'aris'.
make_rand_aris_key generates a random 'key' to be used in encoding and decoding an aristocrat. The key is a list of numbers. The index of the number is the letter that codes for another, and the number in the index is the number of the letter that it codes for. For example: index 0 represents 'a'. If the number at index 0 is 1, then whenever an 'a' comes up in a text, it would be encoded into a 'b', because b is the letter at index 1 of alpha_list.

In [64]:
# returns a key, a list of numbers, random permutation of 0-25
def make_rand_aris_key():
    nums = []
    for i in range(26):
        nums.append(i)
    num_key = []
    for i in range(26):
        num = (random.choice(nums))
        num_key.append(num)
        nums.remove(num)
    return num_key

The next two ciphers are similar to the encode and decode caesar ciphers. Instead of just shifting each letter, however, they use the key to encode and decode the letters.

In [65]:
# plaintext: string or list of characters
# num_key is a list of numbers 
# returns a encoded list of characters
def encode_aris(plaintext, num_key):
    ciphertext_num = []
    plaintext_num = char_to_num(plaintext)
    for num in plaintext_num:
        if num != 26:
            num = num_key[num]
        ciphertext_num.append(num)
    ciphertext = num_to_char(ciphertext_num)
    return ciphertext

In [66]:
def decode_aris(ciphertext, num_key):
    plaintext_num = []
    ciphertext_num = char_to_num(ciphertext)
    for num in ciphertext_num:
        if num != 26:
            num = num_key.index(num)
        plaintext_num.append(num)
    plaintext = num_to_char(plaintext_num)
    return plaintext

In [120]:
key = make_rand_aris_key()
ciphertext = encode_aris('hello', key)
plaintext = decode_aris(ciphertext, key)
print(ciphertext)
print(plaintext)

['k', 'h', 'i', 'i', 'y']
['h', 'e', 'l', 'l', 'o']


count_letters takes a text and counts the number of times each letter is used. the list returned has 27 entries, one for each letter in the alphabet, and one for a space. This is then used to generate the frequency of each letter, using the normalize_counts function, which returns a list containing the decimal percent of times each letter comes up. The indices of the percent indicate the letter.

In [67]:
def count_letters(text, pseudo_count = 0):
    letter_count = []
    for i in range(27):
        letter_count.append(pseudo_count)
    for letter in text:
        letter_count[alpha_list.index(letter)] += 1
    return letter_count

In [68]:
letter_count = count_letters('abcdef')
print(letter_count)

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


The next functions are similar but instead of counting the single letters, they count the number of times each pair of letters occurs (including spaced as a letter). Each letter pair is then turned into a decimal. This means that for each letter, the likelihood that it will be followed by the next letter is the likelihood of each pair. For example, anytime you see a 't' there is about a 50% chance the next letter will be an 'h', so the likelihood for 'th' is 0.50...

In [69]:
def count_letter_pairs(text, pseudo_count = 0):
    pair_count = []
    for i in range(27):
        pair_count.append([])
        for l2 in range(27):
            pair_count[i].append(pseudo_count)
    for i in range(len(text) - 1):
        pair_count[alpha_list.index(text[i])][alpha_list.index(text[i + 1])] += 1
    return pair_count

In [70]:
print(count_letter_pairs('aaaaabbbbbcccccddddd'))

[[4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0,

In [71]:
def normalize_counts(count_list):
    total = 0
    normalized_counts = []
    for i in range(27):
        total += count_list[i]
        normalized_counts.append(0)
    for i in range(27):
        normalized_counts[i] = count_list[i] / total
    return normalized_counts

In [72]:
def normalize_counts_no_spaces(count_list):
    total = 0
    normalized_counts = []
    for i in range(26):
        total += count_list[i]
        normalized_counts.append(0)
    for i in range(26):
        normalized_counts[i] = count_list[i] / total
    return normalized_counts

In [73]:
print(len(normalize_counts_no_spaces(letter_count)))


26


In [74]:
def compute_transition_matrix(pair_count):
    transition_matrix = []
    for i in range(27):
        transition_matrix.append([])
    for i in range(27):
        transition_matrix[i] = (normalize_counts(pair_count[i]))
    return transition_matrix

In [75]:
def find_likelihood(text, percent_list):
    likely = 1
    for letter in text:
        letter = alpha_list.index(letter)
        likely *= percent_list[letter]
    return likely

In [76]:
def find_log_likelihood(text, percent_list):
    log_likely = 0
    for letter in text:
        letter = alpha_list.index(letter)
        log_likely += log(percent_list[letter])
    return log_likely

In [77]:
def find_pair_log_likelihood(text, ref_percent_list, ref_transition_matrix):
    log_likely = 0
    log_likely += log(ref_percent_list[alpha_list.index(text[0])])
    for letter in range(len(text) - 1):
        first_index = alpha_list.index(text[letter])
        second_index = alpha_list.index(text[letter + 1])
        if transition_matrix[first_index][second_index] != 0:
            log_likely += log(transition_matrix[first_index][second_index])
    return log_likely

In [78]:
#tells you how much the log likelihood will change if you switch the letters at the two indices
def log_likelihood_change(text, pair_count, transition_matrix, start_key, index1, index2):
    changed_list = list(transition_matrix)
    unchanged_rows, changed_rows, unchanged_columns, changed_columns = 0, 0, 0, 0
    key1 = start_key[index1]
    #print(key1)
    key2 = start_key[index2]
    #print(key2)
    changed_list[key1], changed_list[key2] = list(transition_matrix[key2]), list(transition_matrix[key1])
    for i in range(27):
        unchanged_rows += log(transition_matrix[key1][i]) * pair_count[key1][i]
        unchanged_rows += log(transition_matrix[key2][i]) * pair_count[key2][i]
        changed_rows += log(changed_list[key1][i]) * pair_count[key1][i]
        changed_rows += log(changed_list[key2][i]) * pair_count[key2][i]
    for i in range(27):
        changed_list[i][key1], changed_list[i][key2] = float(transition_matrix[i][key2]), float(transition_matrix[i][key1])
        unchanged_columns += (log(transition_matrix[i][key1]) * pair_count[i][key1])
        unchanged_columns += (log(transition_matrix[i][key2]) * pair_count[i][key2])
        changed_columns += (log(changed_list[i][key1]) * pair_count[i][key1])
        changed_columns += (log(changed_list[i][key2]) * pair_count[i][key2])
    change = changed_rows + changed_columns - unchanged_rows - unchanged_columns
    return change
        
        

In [189]:
def calculate_likelihood_change(text, text_pair_count, ref_letter_percent,\
                                ref_transition_matrix, index1, index2, key):
    column_change, row_change = 0, 0
    l1 = int(key[index1])
    l2 = int(key[index2])
    for j in range(27):
        if j != l1 and j != l2:
            column_change += (text_pair_count[l1][j] - text_pair_count[l2][j])\
            * (log(ref_transition_matrix[l1][j]) - log(ref_transition_matrix[l2][j]))
    for i in range(27):
        if i != l1 and i != l2:
            row_change += (text_pair_count[i][l1] - text_pair_count[i][l2])\
            * (log(ref_transition_matrix[i][l1]) - log(ref_transition_matrix[i][l2]))
    
    change = column_change + row_change
    
    change += (text_pair_count[l1][l1] - text_pair_count[l2][l2])\
    * (log(ref_transition_matrix[l1][l1]) - log(ref_transition_matrix[l2][l2]))
    
    change += (text_pair_count[l1][l2] - text_pair_count[l2][l1])\
    * (log(ref_transition_matrix[l1][l2]) - log(ref_transition_matrix[l2][l1]))
    #dealing with the first letter (special case)
    
    if text[0] == alpha_list[l1]:
        change += log(ref_letter_percent[l1]) - log(ref_letter_percent[l2])
    elif text[0] == alpha_list[l2]:
        change += log(ref_letter_percent[l2]) - log(ref_letter_percent[l1])
    return -1 * change
    
    

In [80]:
bible_letter_count = count_letters(bible_letters, 0.5)
bible_letter_percent = normalize_counts_no_spaces(bible_letter_count)

bible_pair_count = count_letter_pairs(bible_letters, 0.5)
bible_matrix = compute_transition_matrix(bible_pair_count)

In [180]:
test_text = 'hallo how era you'
changed_text = 'hello how are you'
text_letter_count = count_letters(test_text)
text_pair_count = count_letter_pairs(test_text)



one = find_pair_log_likelihood( test_text, bible_letter_percent, bible_matrix)
two = find_pair_log_likelihood(changed_text, bible_letter_percent, bible_matrix)
diff = (two - one)

start_key = list(alpha_list)
num_key = char_to_num(start_key)
index1 = alpha_list.index("a")
index2 = alpha_list.index("e")


faster_calc = (calculate_likelihood_change(test_text, text_pair_count, bible_letter_percent, bible_matrix,\
                                  index1, index2, num_key))
print(faster_calc)

assert isclose(diff, faster_calc), 'should be equal'
print("test passed")

5.411230978190019
test passed


In [82]:
def find_letter_num(text, letter_count):
    return len(text) - letter_count[26]

In [83]:
def find_solution_brute(ciphertext, percent_list):
    solutions = []
    for i in range(26):
        plaintext = decode_caesar_cipher(ciphertext, i)
        plaintext = ''.join(plaintext)
        likely = find_log_likelihood(plaintext, percent_list)
        solutions.append(likely)
    return solutions

In [84]:
def find_solution_brute_pairs(ciphertext, new_dict = {}, both_list = []):
    solutions = []
    for i in range(26):
        plaintext = decode_caesar_cipher(ciphertext, i)
        plaintext = ''.join(plaintext)
        likely = find_pair_log_likelihood(plaintext, percent_list, transition_matrix)
        solutions.append(likely)
    return solutions

In [85]:
def ordered_solutions(ciphertext, function_name = find_solution_brute_pairs):
    solution_list = []
    solutions = (function_name(ciphertext, percent_list))
    index_array = numpy.flipud(numpy.argsort(solutions))
    for i in range(26):
        solution_list.append("%02d: %s: %s" % ((index_array[i]), ''.join(decode_caesar_cipher(ciphertext, index_array[i])), str(solutions[index_array[i]])))
    return solution_list

In [86]:
def make_letters(file_name):
    new_file = open('../data/' + file_name, 'r')
    file = new_file.read()
    file = file.lower()
    #regular expressions is imported
    #The next line takes out only the letters
    return re.findall('[a-z]|\ ', file)

In [87]:
def get_percent_lists(text_letters):
    letter_count = count_letters(text_letters)
    letter_num = find_letter_num(text_letters, letter_count)
    percent_list = normalize_counts_no_spaces(letter_count)
    pair_count = count_letter_pairs(text_letters)
    transition_matrix = compute_transition_matrix(pair_count)
    return percent_list, transition_matrix

In [88]:
def find_language(text, german_percent_list, german_transition_matrix, english_percent_list, english_transition_matrix):
    german_likelihood = find_pair_log_likelihood(text, german_percent_list, german_transition_matrix)
    english_likelihood = find_pair_log_likelihood(text, english_percent_list, english_transition_matrix)
    if german_likelihood > english_likelihood:
        return 'The text is most likely German'
    else:
        return 'The text is most likely English'

In [89]:
german_letters = make_letters('german_text.txt')
german_list, german_pair_count = get_percent_lists(german_letters)
english_letters = make_letters('bible.txt')
english_list, english_pair_count = get_percent_lists(english_letters)
print(find_language('hallo wie gehts du', german_list, german_pair_count, english_list, english_pair_count))

The text is most likely German


In [90]:
def decode_aris_no_key(ciphertext, start_key, guess_num = 3000):
    count = 0
    num_key = start_key
    new_num_key = list(num_key)
    for i in range(guess_num):
        index1 = random.randint(0, 25)
        index2 = random.randint(0, 25)
        plaintext = decode_aris(ciphertext, num_key)
        new_num_key[index1], new_num_key[index2] = num_key[index2], num_key[index1]
        new_plaintext = decode_aris(ciphertext, new_num_key)
        plaintext_likely = find_pair_log_likelihood(plaintext, percent_list, transition_matrix)
        new_plaintext_likely = find_pair_log_likelihood(new_plaintext, percent_list, transition_matrix)
        if new_plaintext_likely > plaintext_likely:
            num_key = list(new_num_key)
            pair_count[index1], pair_count[index2] = pair_count[index2], pair_count[index1]
            count += 1
        else:
            new_num_key = list(num_key)
        plaintext = decode_aris(ciphertext, num_key)
        log_likelihood = find_pair_log_likelihood(plaintext, percent_list, transition_matrix)
    return (''.join(decode_aris(ciphertext, num_key))), count, log_likelihood

In [158]:
def switch_row_and_columns(text_pair_count, index1, index2):
    text_1 = list(text_pair_count[index1])
    text_pair_count[index1] = list(text_pair_count[index2])
    text_pair_count[index2] = list(text_1)
    for i in range(len(text_pair_count)):
        value_i1 = int(text_pair_count[i][index1])
        value_i2 = int(text_pair_count[i][index2])
        text_pair_count[i][index1] = value_i2
        text_pair_count[i][index2] = value_i1
    return text_pair_count
        

In [213]:
def decode_aris_faster(ciphertext, start_key, text_pair_count, ref_letter_percent, \
                       ref_transition_matrix, guess_num = 3000):
    count = 0
    num_key = list(start_key)
    for i in range(guess_num):
        index1 = random.randint(0, 25)
        index2 = random.randint(0, 25)
        while index1 == index2:
            index1 = random.randint(0, 25)
        plaintext = decode_aris(ciphertext, num_key)
        change = calculate_likelihood_change(plaintext, text_pair_count, ref_letter_percent,\
                                             ref_transition_matrix, index1, index2, num_key)
        print(change)
        if change > 0:
            num_1 = int(num_key[index1])
            num_key[index1] = int(num_key[index2])
            num_key[index2] = num_1
            count += 1
            print(count)
            text_pair_count = switch_row_and_columns(text_pair_count, index1, index2)
    return(''.join(decode_aris(ciphertext, num_key)))
        

In [214]:
ciphertext = encode_aris(eman_letters, make_rand_aris_key())
text_pair_count = count_letter_pairs(ciphertext)
letter_count = count_letters(bible_letters)
ref_pair_count = count_letter_pairs(bible_letters, 0.5)
ref_percent_list = normalize_counts_no_spaces(letter_count)
ref_transition_matrix = compute_transition_matrix(ref_pair_count)
start_key = make_start_key(ciphertext, ref_percent_list)

In [215]:
print(''.join(decode_aris(ciphertext, start_key)))
plaintext = (decode_aris_faster(ciphertext, start_key, text_pair_count, ref_percent_list, \
                        ref_transition_matrix))
#print(find_pair_log_likelihood(plaintext, ref_percent_list, ref_transition_matrix))
#print(find_pair_log_likelihood(eman_letters, ref_percent_list, ref_transition_matrix))

jhamhsw  h tshanusiytioabw tre ysenideat of tre maited nthten of hgesiuhh ysoulhghtioapresehn oa tre tpeatwneuoad dhw of neytegbes ia tre wehs of oms losd oae tromnhad eicrt rmadsed had niktwtpo h ysoulhghtioa phn innmed bw tre ysenideat of tre maited nthten uoathiaiac hgoac otres triacn tre follopiac to pittrht oa tre fisnt dhw of jhamhsw ia tre wehs of oms losd oae tromnhad eicrt rmadsed had niktwtrsee hll yesnoan reld hn nlhven pitria haw nthte os denicahted yhst of h nthte tre yeoyle preseof nrhll trea be ia sebellioa hchiant tre maited nthten nrhll be trea treauefosphsd had foseves fsee had tre ekeumtive covesageat of tre maited nthten iaulmdiac tre gilithsw had ahvhl hmtrositw treseof pill seuocaixe had ghiathia tre fseedog of nmur yesnoan had pill do ao hut os hutn to seysenn nmur yesnoan os haw of treg ia haw effostn trew ghw ghze fos treis hutmhl fseedogtrht tre ekeumtive pill oa tre fisnt dhw of jhamhsw hfosenhid bw ysoulhghtioa denicahte tre nthten had yhstn of nthten if haw

In [201]:
plaintext

'tjrhjea  j bejrgieksbkqrfa buc secgklcrb qo buc hrkbcl gbjbcg qo jdcekijj seqinjdjbkqrxucecjg qr buc bxcrbagciqrl lja qo gcsbcdfce kr buc acje qo qhe nqel qrc buqhgjrl ckzub uhrlecl jrl gkybabxq j seqinjdjbkqr xjg kgghcl fa buc secgklcrb qo buc hrkbcl gbjbcg iqrbjkrkrz jdqrz qbuce bukrzg buc oqnnqxkrz bq xkbbujb qr buc okegb lja qo tjrhjea kr buc acje qo qhe nqel qrc buqhgjrl ckzub uhrlecl jrl gkybabuecc jnn scegqrg ucnl jg gnjpcg xkbukr jra gbjbc qe lcgkzrjbcl sjeb qo j gbjbc buc scqsnc xucecqo gujnn bucr fc kr ecfcnnkqr jzjkrgb buc hrkbcl gbjbcg gujnn fc bucr bucricoqexjel jrl oqecpce oecc jrl buc cycihbkpc zqpcerdcrb qo buc hrkbcl gbjbcg krinhlkrz buc dknkbjea jrl rjpjn jhbuqekba bucecqo xknn eciqzrkvc jrl djkrbjkr buc oecclqd qo ghiu scegqrg jrl xknn lq rq jib qe jibg bq ecsecgg ghiu scegqrg qe jra qo bucd kr jra cooqebg buca dja djmc oqe bucke jibhjn oecclqdbujb buc cycihbkpc xknn qr buc okegb lja qo tjrhjea joqecgjkl fa seqinjdjbkqr lcgkzrjbc buc gbjbcg jrl sjebg qo gbjbcg ko jr

In [210]:
print(find_pair_log_likelihood(plaintext, ref_percent_list, ref_transition_matrix))
print(find_pair_log_likelihood(decode_aris(eman_letters, start_key), ref_percent_list, ref_transition_matrix))

-24122.36205827948
-20877.273480721084


In [94]:
'''
right = 0
wrong = 0
eman_letters = make_letters('Emancipation_Proclamation.txt')
ciphertext = (encode_aris(eman_letters, make_rand_aris_key()))
bible_letters = make_letters('bible.txt')
bible_percent_list, bible_transition_matrix = get_percent_lists(bible_letters)
for i in range(100):
    decoded_text, count, log_likelihood = decode_aris_no_key(ciphertext, make_start_key(ciphertext, bible_percent_list))
    if (log_likelihood) == (find_pair_log_likelihood(eman_letters, percent_list, transition_matrix)):
        right += 1
    else:
        wrong += 1
print('right: ' + str(right))
'''

"\nright = 0\nwrong = 0\neman_letters = make_letters('Emancipation_Proclamation.txt')\nciphertext = (encode_aris(eman_letters, make_rand_aris_key()))\nbible_letters = make_letters('bible.txt')\nbible_percent_list, bible_transition_matrix = get_percent_lists(bible_letters)\nfor i in range(100):\n    decoded_text, count, log_likelihood = decode_aris_no_key(ciphertext, make_start_key(ciphertext, bible_percent_list))\n    if (log_likelihood) == (find_pair_log_likelihood(eman_letters, percent_list, transition_matrix)):\n        right += 1\n    else:\n        wrong += 1\nprint('right: ' + str(right))\n"

In [ ]:
print('right: ' + str(right))

In [ ]:
bible_letters = make_letters('bible.txt')
crime_letters = make_letters('crime_and_punishment.txt')

#For the first case, text = bible_letters for all
letter_count = count_letters(bible_letters)
letter_num = find_letter_num(bible_letters, letter_count)
percent_list = normalize_counts_no_spaces(letter_count)
pair_count = count_letter_pairs(bible_letters)
transition_matrix = compute_transition_matrix(pair_count)

In [ ]:
#print(transition_matrix)
print(transition_matrix[alpha_list.index(" ")][alpha_list.index(" ")])
print(sum(pair_count[alpha_list.index('s')]))
#0.0003554540226212538

In [ ]:
plaintext = 'queen'
plaintext = (plaintext.lower())
ciphertext = encode_caesar_cipher(plaintext, 3)
solutions = []
#print(find_solution_brute(ciphertext))
solutions = (find_solution_brute_pairs(ciphertext))

In [ ]:
caesar_ciphertext = "alex"
solution_list = ordered_solutions(caesar_ciphertext, find_solution_brute)
print("Solutions using single letter frequencies")
for i in range(26):
    print(solution_list[i])
solution_list = ordered_solutions(caesar_ciphertext)
print("Solutions using letter pair frequencies")
for i in range(26):
    print(solution_list[i])

In [168]:
def make_start_key(text, ref_percent_list):
    start_key = []
    text_letter_count = count_letters(text)
    text_percent_list = normalize_counts_no_spaces(text_letter_count)
    sorted_text_percent_list = list(text_percent_list)
    sorted_text_percent_list.sort(reverse = True)
    sorted_ref_percent_list = list(ref_percent_list)
    sorted_ref_percent_list.sort(reverse = True)
    for i in range(26):
        start_key.append(0)
    for i in range(26):
        key_value = ref_percent_list.index(sorted_ref_percent_list[i])
        key_index = text_percent_list.index(sorted_text_percent_list[i])
        start_key[key_index] = key_value
        text_percent_list[key_index] = -1
    return start_key

In [170]:
eman_letters = make_letters('Emancipation_Proclamation.txt')
#ciphertext = encode_aris(eman_letters, make_rand_aris_key())
ref_letter_count = count_letters(bible_letters, 0.5)
ref_letter_num = find_letter_num(bible_letters, ref_letter_count)
ref_percent_list = normalize_counts_no_spaces(ref_letter_count)
start_key = make_start_key(eman_letters, ref_percent_list)
right = 0
wrong = 0
plaintext = (''.join(decode_aris(ciphertext, start_key)))
print(start_key)
#print(plaintext)

[13, 1, 6, 3, 4, 5, 12, 0, 8, 23, 9, 11, 20, 18, 14, 22, 16, 7, 17, 19, 2, 21, 24, 25, 15, 10]
sfalfwn  f jwfazmwrtjrqaon jub twbzrebaj qg jub larjbe zjfjbz qg fdbwrmff twqmhfdfjrqayubwbfz qa jub jybajnzbmqae efn qg zbtjbdobw ra jub nbfw qg qlw hqwe qab juqlzfae brpuj ulaewbe fae zrijnjyq f twqmhfdfjrqa yfz rzzlbe on jub twbzrebaj qg jub larjbe zjfjbz mqajfrarap fdqap qjubw jurapz jub gqhhqyrap jq yrjjufj qa jub grwzj efn qg sfalfwn ra jub nbfw qg qlw hqwe qab juqlzfae brpuj ulaewbe fae zrijnjuwbb fhh tbwzqaz ubhe fz zhfxbz yrjura fan zjfjb qw ebzrpafjbe tfwj qg f zjfjb jub tbqthb yubwbqg zufhh juba ob ra wbobhhrqa fpfrazj jub larjbe zjfjbz zufhh ob juba jubambgqwyfwe fae gqwbxbw gwbb fae jub bibmljrxb pqxbwadbaj qg jub larjbe zjfjbz ramhlerap jub drhrjfwn fae afxfh fljuqwrjn jubwbqg yrhh wbmqparkb fae dfrajfra jub gwbbeqd qg zlmu tbwzqaz fae yrhh eq aq fmj qw fmjz jq wbtwbzz zlmu tbwzqaz qw fan qg jubd ra fan bggqwjz jubn dfn dfvb gqw jubrw fmjlfh gwbbeqdjufj jub bibmljrxb yrhh qa jub

In [129]:
len(ref_percent_list)

26

In [ ]:
print(start_key)
print(start_key.index(16))